### 10.3.3 Scrape Mars Data: The News

In [ ]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoupc
import pandas as pd

In [ ]:
# Set the executable path and initialize the chrome browser in splinter
executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path)

In [4]:
# Visit the mars nasa news site
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

True

In [5]:
# set up the HTML parser:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
slide_elem = soup.select_one('ul.item_list li.slide')

#### Begin Scraping

In [6]:
# Assign the title and summary text to variables - will reference later
slide_elem.find("div", class_='content_title')

<div class="content_title"><a href="/news/8719/nasa-invites-public-to-share-excitement-of-mars-2020-perseverance-rover-launch/" target="_self">NASA Invites Public to Share Excitement of Mars 2020 Perseverance Rover Launch</a></div>

In [7]:
# The title is in that mix of HTML in our output
# But we need to get just the text, and the extra HTML stuff isn’t necessary. 

# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find("div", class_='content_title').get_text()
news_title

# This returns the most recent title published

'NASA Invites Public to Share Excitement of Mars 2020 Perseverance Rover Launch'

In [8]:
# This gets the summary info
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_="article_teaser_body").get_text()
news_p

'There are lots of ways to participate in the historic event, which is targeted for July 30.'

##### 10.3.4 Scrape Mars Data: Featured Image

In [9]:
# Visit URL
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [10]:
# Find and click the full image button
full_image_elem = browser.find_by_id('full_image')
full_image_elem.click()

In [11]:
# Find the more info button and click that
browser.is_element_present_by_text('more info', wait_time=1)
more_info_elem = browser.links.find_by_partial_text('more info')
more_info_elem.click()

In [13]:
# Parse the resulting html with soup
html = browser.html
img_soup = BeautifulSoup(html, 'html.parser')

In [14]:
# Find the relative image url
img_url_rel = img_soup.select_one('figure.lede a img').get("src")
img_url_rel

'/spaceimages/images/largesize/PIA17462_hires.jpg'

In [15]:
# Use the base URL to create an absolute URL
img_url = f'https://www.jpl.nasa.gov{img_url_rel}'
img_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17462_hires.jpg'

##### 10.3.5 Scrape Mars Data: Mars Facts

In [19]:
html_df = pd.read_html('http://space-facts.com/mars/')[0]
html_df.columns=['description', 'value']
html_df.set_index('description', inplace=True)
html_df

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [20]:
# convert DataFrame back into HTML-ready code
html_df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\

##### Challenge scrape

In [4]:
# Go to the page with hemisphere images
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=2)

False

In [6]:
# Get the total number of images to loop through
total_images = len(browser.find_by_css("a.product-item h3"))
total_images

4

In [7]:
# List to hold hemisphere dictionaries
Hemisphere_list =[]

In [9]:
# For loop to get all the image info
for x in range(total_images):
    # Challenge instruction #1
    # Get the first image
    full_image_elem = browser.find_by_css('a.product-item h3')[x]
    full_image_elem.click()
    
    # Parse
    html = browser.html
    img_soup = soup(html, 'html.parser')

    # Find the relative image url
    img_url_rel = img_soup.select_one('div.container div img.wide-image').get("src")
    
    # Challenge instruction #2
    # Create full URL and get the title
    img_url = f'https://www.jpl.nasa.gov{img_url_rel}'

    img_title = img_soup.select_one("div.container div h2.title").get_text()

    
    # Challenge instruction #3
    # Store url & title in a dictionary
    hemi_dict = {
        "img_url": img_url,
        "title": img_title
    }

    # Challenge instruction #4
    # append the dictionary to the Hemisphere_list (list of dictionaries)
    Hemisphere_list.append(hemi_dict)
    
    # got back to the list of images
    browser.back()

https://www.jpl.nasa.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg
Cerberus Hemisphere Enhanced
https://www.jpl.nasa.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg
Schiaparelli Hemisphere Enhanced
https://www.jpl.nasa.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg
Syrtis Major Hemisphere Enhanced
https://www.jpl.nasa.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg
Valles Marineris Hemisphere Enhanced


In [25]:
# Scrape completed
browser.quit()

#### Scrape completed

### 10.3.6 Export to Python